In [1]:
import os
from PIL import Image
from torchvision import transforms
import numpy as np
import cv2 as cv
import torch

In [2]:
data_root = '../data/img_residential_original/'
all_data_dirs = os.listdir(data_root)
all_data_dirs = [data_root + str(i) + '.png' for i in range(0,len(all_data_dirs))]

In [189]:
class Centeralize(object):
    """Crop randomly the image in a sample.

    Args:
        output_size (tuple): Desired output size.
    """

    def __init__(self, output_size):
        assert isinstance(output_size, (int))
        self.output_size = output_size

    def __call__(self, image):
        tmp = image
        image = torch.sum(image, dim=0)/3
        h, w = image.shape
        half_h = half_w = int(self.output_size/2)
        grid_x = torch.FloatTensor([[i for i in range(0,w)] for j in range(0,h)])
        grid_y = torch.FloatTensor([[j for i in range(0,w)] for j in range(0,h)])
        img_reverse = (image<0.98).float()
        center_w = int(torch.sum(img_reverse*grid_x)/torch.sum(img_reverse))
        center_h = int(torch.sum(img_reverse*grid_y)/torch.sum(img_reverse))
        print(center_h,center_w)

        top = center_h - half_h
        left = center_w - half_w
        

        image = tmp[:, top: top + self.output_size,
                      left: left + self.output_size]

        return image

In [190]:
composed = transforms.Compose([
                                Centeralize(1000),
                                transforms.Normalize((0.5,0.5,0.5), (1.0,1.0,1.0)),
                                transforms.Grayscale(1),
                                transforms.Resize(112),
                                transforms.CenterCrop(56),
                              ])
def saveto(i):
    img = Image.open(all_data_dirs[i])
    img = np.array(img)/255.0
    img = np.transpose(img[:, :, :3], (2, 0, 1))
    img_tensor = torch.from_numpy(img.astype(np.float32))
    return composed(img_tensor)

In [191]:
grid_x = [[i for i in range(0,5)] for j in range(0,3)]
grid_y = [[j for i in range(0,5)] for j in range(0,3)]
print(grid_x,grid_y)

[[0, 1, 2, 3, 4], [0, 1, 2, 3, 4], [0, 1, 2, 3, 4]] [[0, 0, 0, 0, 0], [1, 1, 1, 1, 1], [2, 2, 2, 2, 2]]


In [192]:
saveto(0)

637 1331


tensor([[[0.5000, 0.4999, 0.5000,  ..., 0.4529, 0.5000, 0.5000],
         [0.4999, 0.5000, 0.5000,  ..., 0.4529, 0.4999, 0.4999],
         [0.5000, 0.4999, 0.4999,  ..., 0.4529, 0.1754, 0.4999],
         ...,
         [0.5000, 0.4999, 0.4529,  ..., 0.5000, 0.4999, 0.5000],
         [0.5000, 0.4999, 0.4529,  ..., 0.4999, 0.4999, 0.5000],
         [0.5000, 0.4999, 0.4529,  ..., 0.5000, 0.5000, 0.5000]]])

In [14]:
contours

[array([[[1553,  353]],
 
        [[1554,  352]],
 
        [[1558,  352]],
 
        ...,
 
        [[1542,  354]],
 
        [[1547,  354]],
 
        [[1548,  353]]], dtype=int32),
 array([[[   0,    0]],
 
        [[   0, 1256]],
 
        [[2448, 1256]],
 
        [[2448,    0]]], dtype=int32)]

In [4]:
saveto(0)

tensor([[[0.4999, 0.4999, 0.4999,  ..., 0.4999, 0.4999, 0.4999],
         [0.4999, 0.4999, 0.4999,  ..., 0.4999, 0.4999, 0.4999],
         [0.4999, 0.4999, 0.5000,  ..., 0.5000, 0.5000, 0.4999],
         ...,
         [0.4999, 0.4999, 0.5000,  ..., 0.5000, 0.5000, 0.4999],
         [0.4999, 0.4999, 0.5000,  ..., 0.5000, 0.5000, 0.4999],
         [0.4999, 0.4999, 0.4999,  ..., 0.4999, 0.4999, 0.4999]]])

In [18]:

torch.save(saveto(0), '../data/img_residential_original/0.pt')